# 🧪 Finance ABAC Demo - Step 4: Test ABAC Policies

## 📋 Overview
This notebook defines ABAC policies using the functions created in Step 1 and tests them using the datasets created in Step 2. Each policy uses the account users group that is defined by default.

### What This Notebook Does:
1. **Tests Each Function**: Runs before/after examples for every masking function
2. **Validates Output**: Ensures masked data meets requirements
3. **Demonstrates Usage**: Shows how to apply functions in real queries

### Why Test Functions?
Testing ensures:
- **Correctness**: Functions work as designed
- **Data Integrity**: Original data isn't corrupted
- **Performance**: Functions execute efficiently
- **Compliance**: Masking meets regulatory requirements
- **User Experience**: Masked output is appropriate for different roles

### What You'll See:
For each function, you'll see:
- **Original Data**: Unmasked values from tables
- **Masked/Filtered Data**: Transformed values after function application
- **Side-by-Side Comparison**: Before and after for easy validation

## 🎓 How to Use This Notebook
1. **Ensure Steps 1-3 Complete**: All functions, tables, and data must exist
2. **Run All Cells**: Execute sequentially to see all test results
3. **Review Output**: Compare original vs masked data
4. **Verify Expectations**: Check that masking behavior is appropriate

## ⚙️ Prerequisites
- ✅ **Step 1 completed**: All functions created
- ✅ **Step 2 completed**: Core tables with data
- ✅ **Step 3 completed**: Governed tags defined and assigned
- ✅ SELECT permission on all tables and functions

## 📊 Expected Results
After running this notebook:
- ✅ Masking/filtering functions tested with ABAC policies on the finance schema
- ✅ Before/after comparisons shown
- ✅ Confidence that ABAC setup is working correctly
---


In [0]:
pip install pyyaml

In [0]:
# 📋 Load Configuration from config.yaml
import yaml
from pathlib import Path

config_file = Path('config.yaml')
if config_file.exists():
    with open(config_file) as f:
        config = yaml.safe_load(f)
    CATALOG = config['catalog']
    SCHEMA = config['schema']
    print(f'✅ Configuration loaded from config.yaml')
    print(f'   📊 Catalog: {CATALOG}')
    print(f'   📁 Schema: {SCHEMA}')
else:
    # Fallback defaults
    CATALOG = 'your_catalog_name'
    SCHEMA = 'finance'
    print(f'⚠️  config.yaml not found - using defaults')
    print(f'   📊 Catalog: {CATALOG}')
    print(f'   📁 Schema: {SCHEMA}')

# Set catalog and schema to use in following cells
spark.sql(f"USE CATALOG {CATALOG}")
spark.sql(f"USE SCHEMA {SCHEMA}")
spark.sql(f"DECLARE VARIABLE target_schema = '{SCHEMA}'")


## ⚙️ Configuration

Testing functions in:
- **Catalog**: `your_catalog_name`
- **Schema**: `finance`


In [0]:
%sql
SELECT '🧪 Testing functions in: ' || current_catalog() || '.' || current_schema() AS status;

## Query tables before applying any policies


In [0]:
%sql
SELECT * FROM customers

In [0]:
%sql
SELECT * FROM accounts

In [0]:
%sql
SELECT * FROM credit_cards

In [0]:
%sql
SELECT * FROM transactions

In [0]:
%sql
SELECT * FROM loans

## Test: CREDIT CARD MASKING (PCI-DSS)


In [0]:
spark.sql(f"""
  CREATE POLICY card_mask ON SCHEMA {SCHEMA}
  COLUMN MASK mask_credit_card 
  TO `account users`
  FOR TABLES
  MATCH COLUMNS
    hasTagValue('pii_type','credit_card') AND hasTag('pci_compliance') AS card
  ON COLUMN card""")

In [0]:
%sql
-- =============================================
-- TEST 1: Credit Card Masking Demo
-- =============================================

SELECT 
  card_id,
  customer_id,
  card_number,
  card_type
FROM credit_cards
LIMIT 5;

In [0]:
spark.sql(f"""DROP POLICY card_mask ON SCHEMA {SCHEMA}""")

## Test: ACCOUNT NUMBER MASKING


In [0]:
spark.sql(f"""
  CREATE POLICY account_mask ON SCHEMA {SCHEMA}
  COLUMN MASK mask_account_last4 
  TO `account users`
  FOR TABLES
  MATCH COLUMNS
    hasTagValue('pii_type','account') AND hasTagValue('data_classification','Confidential') AS account
  ON COLUMN account""")

In [0]:
%sql
-- =============================================
-- TEST 2: Account Number Masking Demo
-- =============================================

SELECT 
  account_id,
  customer_id,
  account_number,
  account_type
FROM accounts
LIMIT 5;

In [0]:
spark.sql(f"""DROP POLICY account_mask ON SCHEMA {SCHEMA}""")

## Test: SSN MASKING


In [0]:
spark.sql(f"""
  CREATE POLICY ssn_mask ON SCHEMA {SCHEMA}
  COLUMN MASK mask_ssn_last4 
  TO `account users`
  FOR TABLES
  MATCH COLUMNS
    hasTagValue('pii_type','ssn') AS ssn
  ON COLUMN ssn""")

In [0]:
%sql
-- =============================================
-- TEST 3: SSN Masking Demo
-- =============================================

SELECT 
  customer_id,
  CONCAT(first_name, ' ', last_name) AS name,
  ssn,
  credit_score
FROM customers
LIMIT 5;

In [0]:
spark.sql(f"""DROP POLICY ssn_mask ON SCHEMA {SCHEMA}""")

## Test: EMAIL & PHONE MASKING


In [0]:
spark.sql(f"""
  CREATE POLICY email_mask ON SCHEMA {SCHEMA}
  COLUMN MASK mask_email 
  TO `account users`
  FOR TABLES
  MATCH COLUMNS
    hasTagValue('pii_type','email') AS email
  ON COLUMN email""")

In [0]:
spark.sql(f"""
  CREATE POLICY phone_mask ON SCHEMA {SCHEMA}
  COLUMN MASK mask_phone 
  TO `account users`
  FOR TABLES
  MATCH COLUMNS
    hasTagValue('pii_type','phone') AS phone
  ON COLUMN phone""")

In [0]:
%sql
-- =============================================
-- TEST 4: Email & Phone Masking Demo
-- =============================================

SELECT 
  customer_id,
  first_name,
  email,
  phone
FROM customers
LIMIT 5;

In [0]:
spark.sql(f"""DROP POLICY email_mask ON SCHEMA {SCHEMA}""")
spark.sql(f"""DROP POLICY phone_mask ON SCHEMA {SCHEMA}""")

## Test: AMOUNT BUCKETING


In [0]:
spark.sql(f"""
  CREATE POLICY transaction_bucket ON SCHEMA {SCHEMA}
  COLUMN MASK mask_amount_bucket 
  TO `account users`
  FOR TABLES
  MATCH COLUMNS
    hasTagValue('pii_type','transaction_amount') AS amount
  ON COLUMN amount""")

In [0]:
%sql
-- =============================================
-- TEST 5: Transaction Amount Bucketing
-- =============================================

SELECT 
  transaction_id,
  amount,
  merchant_name,
  transaction_type
FROM transactions
LIMIT 10;

In [0]:
spark.sql(f"""DROP POLICY transaction_bucket ON SCHEMA {SCHEMA}""")

## Test: ROUTING NUMBER MASKING


In [0]:
spark.sql(f"""
  CREATE POLICY routing_num_mask ON SCHEMA {SCHEMA}
  COLUMN MASK mask_routing_number 
  TO `account users`
  FOR TABLES
  MATCH COLUMNS
    hasTagValue('pii_type','routing_number') AS routing_number
  ON COLUMN routing_number""")

In [0]:
%sql
-- =============================================
-- TEST 6: Routing Number Masking
-- =============================================

SELECT 
  account_id,
  routing_number,
  account_type
FROM accounts
WHERE routing_number IS NOT NULL
LIMIT 5;

In [0]:
spark.sql(f"""DROP POLICY routing_num_mask ON SCHEMA {SCHEMA}""")

## Test: TRANSACTION ID HASHING


In [0]:
spark.sql(f"""
  CREATE POLICY transaction_mask ON SCHEMA {SCHEMA}
  COLUMN MASK mask_transaction_hash 
  TO `account users`
  FOR TABLES
  MATCH COLUMNS
    hasTagValue('pii_type','transaction_id') AS transaction_id
  ON COLUMN transaction_id""")

In [0]:
%sql
-- =============================================
-- TEST 7: Transaction ID Hashing
-- =============================================

SELECT 
  transaction_id,
  amount,
  merchant_name
FROM transactions
LIMIT 5;

In [0]:
spark.sql(f"""DROP POLICY transaction_mask ON SCHEMA {SCHEMA}""")

## Test: CUSTOMER ID MASKING (Deterministic)


In [0]:
spark.sql(f"""
  CREATE POLICY customer_id_mask ON SCHEMA {SCHEMA}
  COLUMN MASK mask_customer_id_deterministic 
  TO `account users`
  FOR TABLES
  MATCH COLUMNS
    hasTagValue('pii_type','id') AS customer_id
  ON COLUMN customer_id""")

In [0]:
%sql
-- =============================================
-- TEST 8: Customer ID Deterministic Masking
-- =============================================

SELECT 
  customer_id,
  first_name,
  last_name
FROM customers
LIMIT 5;

In [0]:
spark.sql(f"""DROP POLICY customer_id_mask ON SCHEMA {SCHEMA}""")

## Test: IP ADDRESS MASKING


In [0]:
spark.sql(f"""
  CREATE POLICY ip_address_mask ON SCHEMA {SCHEMA}
  COLUMN MASK mask_ip_address 
  TO `account users`
  FOR TABLES
  MATCH COLUMNS
    hasTagValue('pii_type','ip_address') AS ip
  ON COLUMN ip""")

In [0]:
%sql
-- =============================================
-- TEST 9: IP Address Masking
-- =============================================

SELECT 
  transaction_id,
  ip_address,
  device_type
FROM transactions
LIMIT 5;

In [0]:
spark.sql(f"""DROP POLICY ip_address_mask ON SCHEMA {SCHEMA}""")

## Test: INCOME BRACKETING


In [0]:
spark.sql(f"""
  CREATE POLICY income_mask ON SCHEMA {SCHEMA}
  COLUMN MASK mask_income_bracket 
  TO `account users`
  FOR TABLES
  MATCH COLUMNS
    hasTagValue('pii_type','income') AS income
  ON COLUMN income""")

In [0]:
%sql
-- =============================================
-- TEST 10: Income Bracket Masking
-- =============================================

SELECT 
  customer_id,
  CONCAT(first_name, ' ', last_name) AS name,
  annual_income
FROM customers
LIMIT 5;

In [0]:
spark.sql(f"""DROP POLICY income_mask ON SCHEMA {SCHEMA}""")

## Test: FRAUD FLAGGED TRANSACTIONS


In [0]:
spark.sql(f"""
  CREATE POLICY fraud_filter ON SCHEMA {SCHEMA}
  ROW FILTER filter_fraud_flagged_only
  TO `account users`
  FOR TABLES
  MATCH COLUMNS
    hasTag('fraud_detection') AS fraud
  USING COLUMNS (fraud)""")

In [0]:
%sql
-- =============================================
-- TEST 11: Fraud Transaction Detection
-- =============================================

SELECT 
  transaction_id,
  customer_id,
  amount,
  merchant_name,
  fraud_flag,
  status
FROM transactions;

In [0]:
spark.sql(f"""DROP POLICY fraud_filter ON SCHEMA {SCHEMA}""")

## Test: HIGH-VALUE TRANSACTIONS (>$5000)


In [0]:
spark.sql(f"""
  CREATE POLICY transactions_filter ON SCHEMA {SCHEMA}
  ROW FILTER filter_high_value_transactions 
  TO `account users`
  FOR TABLES
  MATCH COLUMNS
    hasTagValue('pii_type','transaction_amount') AS transactions
  USING COLUMNS (transactions)""")

In [0]:
%sql
-- =============================================
-- TEST 12: High-Value Transactions 
-- (greater than 5000)
-- =============================================

SELECT 
  transaction_id,
  customer_id,
  amount,
  merchant_name,
  transaction_date
FROM transactions
ORDER BY amount DESC;

In [0]:
spark.sql(f"""DROP POLICY transactions_filter ON SCHEMA {SCHEMA}""")

## Test: BUSINESS HOURS FILTER

In [0]:
spark.sql(f"""
  CREATE POLICY hours_filter ON SCHEMA {SCHEMA}
  ROW FILTER filter_business_hours
  TO `account users`
  FOR TABLES""")

In [0]:
%sql
-- =============================================
-- TEST 13: Business Hours Filter
-- =============================================

SELECT 
  transaction_id,
  customer_id,
  hour(current_timestamp()),
  merchant_name
FROM transactions;

In [0]:
spark.sql(f"""DROP POLICY hours_filter ON SCHEMA {SCHEMA}""")

## Test: STATE ACCESS FILTER

In [0]:
spark.sql(f"""
  CREATE POLICY state_filter ON SCHEMA {SCHEMA}
  ROW FILTER filter_by_region
  TO `account users`
  FOR TABLES
  MATCH COLUMNS
    hasTagValue('pii_type','location') AS state
  USING COLUMNS (state)""")

In [0]:
%sql
-- =============================================
-- TEST 14: Region Access Filter
-- CA only
-- =============================================

SELECT 
  customer_id,
  email,
  state,
  credit_score
FROM customers;

In [0]:
spark.sql(f"""DROP POLICY state_filter ON SCHEMA {SCHEMA}""")

## Cleanup: drop policies + tables

In [0]:
try:
    policies = (spark.sql(f"SHOW POLICIES ON SCHEMA {SCHEMA}")).collect()
    for policy in policies:
        policy_name = policy["Policy Name"]
        try:
            spark.sql(f"DROP POLICY {policy_name} ON SCHEMA {SCHEMA}")
        except Exception as e:
            print(f"⚠️  Could not clean {policy_name}: {str(e)[:40]}")
    print(f"✅ Cleanup complete")
except Exception as e:
    print(f"✅ Cleanup complete")

In [0]:
spark.sql(f"""DROP SCHEMA IF EXISTS {SCHEMA} CASCADE""")

## ✅ All Tests Complete!

Congratulations! All Finance ABAC functions are working correctly!

### What You Verified:
- ✅ All tables contain expected data
- ✅ Masking functions produce correct output
- ✅ Data transformations maintain privacy requirements
- ✅ Functions are ready for ABAC policy integration

### Test Summary:
- **Email Masking**: ✅ Local part hidden, domain visible
- **Phone Masking**: ✅ Showing last 4 digits only
- **Financial Data**: ✅ Bucketed or last-4 protected
- **Identifiers**: ✅ Deterministic hashing working
- **Sensitive Fields**: ✅ Complete redaction successful

### 🎯 Next Steps - Implementing ABAC Policies:

Now that masking functions are tested, you can:

1. **Create User Groups**:
   ```sql
   -- Example: Create groups for different access levels
   CREATE GROUP finance_analysts;
   CREATE GROUP finance_admins;
   ```


2. **Test Policies**:
   - Update ABAC policies to use different groups
   - Query the same table
   - Verify each user sees appropriately masked data

### 📚 Additional Resources:
- [Unity Catalog ABAC Documentation](https://docs.databricks.com/aws/en/data-governance/unity-catalog/abac/)
- [Row Filters and Column Masks](https://docs.databricks.com/aws/en/data-governance/unity-catalog/filters-and-masks)

---
**🎉 Great Job!** Your Finance ABAC demo foundation is complete and tested!
